In [76]:
import requests, json, os #, espnff
import pandas as pd
import numpy as np
from datetime import datetime
league_id = 245118

In [97]:
week = input("Which week is it?")
season = 2023

Which week is it? 1


In [152]:
# code mapping for roster positions
lineupSlotID = {17:'K', 0:'QB',20:'bench', 15:'DP',6:'TE',23:'FLEX', 4:'WR',2:'RB', 21:'IR'}
teamIds = {
    1:'John', 2:'Gomer', 3:'Pope', 4:"Jamie", 5:"Geik", 6:"Bryan", 
    7:"Chaunce", 8:"Sam", 9:"Chris", 10:"Murphy", 11:"Colin", 12:'Ethan'
}

data = {1:{},2:{},3:{},4:{},5:{},6:{},7:{},8:{},9:{},10:{},11:{},12:{}}
for i in range(1,13):
    for j in range(int(week),18): #start range set to week so that the previous weeks don't get erased on accident
        data[i][j] = {'roster':{0:{}, 2:{}, 4:{},  6:{},  23:{},  15:{}, 17:{}, 20:{}, 21:{}},
                      'pf':0, 'pa':0, 
                      'ppf':0, 'ppa':0,  #proj points for and against
                      'pfTotal':0, 'paTotal':0, 
                      'ppfTotal':0, 'ppaTotal':0, 
                      'wl':[0,0,0], #wins, losses, ties
                      'prfScore':0, # the score for the perfect line up
                      'prfLineup':0, #perfect line up, 1 = yes, 0 = no
                      'pwrESPN':0, #espn rank
                      'pwrCommish':0, #commish rank
                      'wastedPts':0, #total points left on the bench, 
                      'scoreRnk':0, #absolute rank of this weeks score 1-12
                      'oppScoreRnk':0, # absolute rank of this weeks score 1-12
                      'projPlusMinus':0, #total pts over or under total projection
                      'oppProjPlusMinus':0, #opp. total pts over or under total projection
                      'oppId':0 #opp id
                     }

In [84]:
boxscores = []
teams = []
players = []

 
directory = r"..\data\Season"
try:
    #looping through every weekly file to aggregate into single df
    for filename in os.listdir(directory):
        f = os.path.join(directory,filename)
        # checking if it is a file
        if os.path.isfile(f):
            with open(f) as json_file:
                js = json.load(json_file)
                    
            if 'boxscore' in filename:
                pass
            elif 'league' in filename:

                season = js['seasonId']
                week = js['scoringPeriodId']
    
                for t in js['teams']:
                    teamcols = [
                            'season', 'week', 'id', 'teamName', 'currentProjectedRank',
                            'draftDayProjectedRank', 'playoffSeed',
                            'rankCalculatedFinal', 'rankFinal', 'waiverRank',
                            'gb', 'wins', 'losses', 'ties', 'wp', 'pf', 'pa', 'streakLength',
                            'streakType', 'budgetSpent', 'acquisitions', 'drops']
                    
                    crank = t['currentProjectedRank']
                    drank = t['draftDayProjectedRank']
                    id = t['id']
                    team = t['location'] + " " + t['nickname']
                    seed = t['playoffSeed']
                    calrank = t['rankCalculatedFinal']
                    finrank = t['rankFinal']
                    waiverrank = t['waiverRank']
                    gb = t['record']['overall']['gamesBack']
                    wins = t['record']['overall']['wins']
                    losses = t['record']['overall']['losses']
                    ties = t['record']['overall']['ties']
                    wp = t['record']['overall']['percentage']
                    pf = t['record']['overall']['pointsFor']
                    pa = t['record']['overall']['pointsAgainst']
                    streak = t['record']['overall']['streakLength']
                    streaktype = t['record']['overall']['streakType']
                    spent = t['transactionCounter']['acquisitionBudgetSpent']
                    acqcount = t['transactionCounter']['acquisitions']
                    drops = t['transactionCounter']['drops']
        
                    temp = [season, week, id, team, crank, drank, seed, calrank, finrank,
                            waiverrank, gb, wins, losses, ties, wp, pf, pa, streak, streaktype, spent,
                        acqcount, drops]
        
                    teams.append(temp)

    ########## this contains all player info in ESPN
            #elif 'player' in filename:
            #    pass    
    
    #boxscores = 
    teams = pd.DataFrame(teams, columns=teamcols).drop_duplicates(['season', 'week', 'id'])
    #players = 
except Exception as e:
    print(e)
    print(t)

In [140]:
week = int(week)
matches = [-6+(6*week),-5+(6*week),-4+(6*week),-3+(6*week),-2+(6*week),-1+(6*week)]
matches

[0, 1, 2, 3, 4, 5]

In [153]:
# loops through the matchups for the week. Matchups are labeled 0 - 83
#  week 1 = 0-5, week 2 = 6-11
week_scores = []
for i in matches:
    #grab data starting point for easier reading later
    game = boxscore_json['schedule'][i]
    #grab team ids for the match up
    away_team = game['away']['teamId']
    home_team = game['home']['teamId']
    data[away_team][week]['oppId'] = home_team
    data[home_team][week]['oppId'] = away_team
    
    #grab base scoring data for easier reading later
    away_stats = game['away']['rosterForCurrentScoringPeriod']
    home_stats = game['home']['rosterForCurrentScoringPeriod']
    
    #update points for and points against for each team
    awayPts = away_stats['appliedStatTotal']
    homePts = home_stats['appliedStatTotal']
    data[away_team][week]['pf'] = awayPts
    data[away_team][week]['pa'] = homePts
    
    #building list of scores to create league weekly ranks
    week_scores.append(awayPts)
    week_scores.append(homePts)
    
    ###data[away_team][week]['pfTotal'] += awayPts
    ###data[away_team][week]['paTotal'] += homePts
    data[home_team][week]['pa'] = awayPts
    data[home_team][week]['pf'] = homePts
    ###data[home_team][week]['paTotal'] += awayPts
    ###data[home_team][week]['pfTotal'] += homePts
    #update record
    if awayPts > homePts:
        data[away_team][week]['wl'][0] += 1
        data[home_team][week]['wl'][1] += 1
    elif awayPts < homePts:
        data[away_team][week]['wl'][1] += 1
        data[home_team][week]['wl'][0] += 1
    else:
        data[away_team][week]['wl'][2] += 1
        data[home_team][week]['wl'][2] += 1
    
    ###################################################
    # looping through home and away data to populate weekly team data
    ##################################################
    ##### AWAY TEAM   
    #populating the player act and proj scores
    for p in away_stats['entries']:
        #set vars for easier reading
        slotId = p['lineupSlotId']
        pId = p['playerId']
        player = p['playerPoolEntry']['player']['fullName']
        defaultSlot = p['playerPoolEntry']['player']['defaultPositionId']
        #set base stat data for easier reading
        stats = p['playerPoolEntry']['player']['stats']
        data[away_team][week]['roster'][slotId][pId] = {}
        data[away_team][week]['roster'][slotId][pId]['name'] = player
        #updating all defensive players to have default slot == 15
        if (defaultSlot >=10) and (defaultSlot <=16):
            data[away_team][week]['roster'][slotId][pId]['defaultSlot'] = 15
        else:
            data[away_team][week]['roster'][slotId][pId]['defaultSlot'] = defaultSlot
        #building the roster performance dictionary - scores, projects, starts, bench
        for s in stats:
            #[statSourceId] == 0 is actual points scored
            if s['statSourceId'] == 0:
                act = s['appliedTotal']
                data[away_team][week]['roster'][slotId][pId]['act'] = act   
            else:
                proj = s['appliedTotal']
                data[away_team][week]['roster'][slotId][pId]['proj'] = proj
                if (slotId == 20) or (slotId == 21):
                    pass
                else:
                    data[away_team][week]['ppf'] += proj

    ## HOME TEAM -same as the loop above but for the home team
    for p in home_stats['entries']:
        #set vars for easier reading
        slotId = p['lineupSlotId']
        pId = p['playerId']
        player = p['playerPoolEntry']['player']['fullName']
        defaultSlot = p['playerPoolEntry']['player']['defaultPositionId']
        #set base stat data for easier reading
        stats = p['playerPoolEntry']['player']['stats']
        data[home_team][week]['roster'][slotId][pId] = {}
        data[home_team][week]['roster'][slotId][pId]['name'] = player
        if (defaultSlot >=10) and (defaultSlot <=16):
            data[home_team][week]['roster'][slotId][pId]['defaultSlot'] = 15
        else:
            data[home_team][week]['roster'][slotId][pId]['defaultSlot'] = defaultSlot 
        for s in stats:
            #[statSourceId] == 0 #actual
            if s['statSourceId'] == 0:
                act = s['appliedTotal']
                data[home_team][week]['roster'][slotId][pId]['act'] = act
            else:
                proj = s['appliedTotal']
                data[home_team][week]['roster'][slotId][pId]['proj'] = proj
                if (slotId == 20) or (slotId == 21):
                    pass
                else:
                    data[home_team][week]['ppf'] += proj    
    
    ###################################
    #exited the home/away loops but still in the game json loop for that week
    ##################################
    
    #calculate PF vs Proj
    data[home_team][week]['ppa'] = data[away_team][week]['ppf']
    data[away_team][week]['ppa'] = data[home_team][week]['ppf']
    data[home_team][week]['projPlusMinus'] = data[home_team][week]['pf'] - data[home_team][week]['ppf']
    data[away_team][week]['projPlusMinus'] = data[away_team][week]['pf'] - data[away_team][week]['ppf']
    data[home_team][week]['oppProjPlusMinus'] = data[away_team][week]['projPlusMinus']
    data[away_team][week]['oppProjPlusMinus'] = data[home_team][week]['projPlusMinus']
    
    #########################
    ### calculating perfect line up scores
    #########################
    
    # creating a dictionary for the away team points by position id
    away_points =  {0:[],2:[],4:[], 6:[], 15:[],17:[]}
    for pos in [0, 2, 4, 6 ,15, 17, 20]:
        for i, p in data[away_team][week]['roster'][pos].items():
            slot = p['defaultSlot']
            if slot == 1:
                away_points[0].append(p['act'])
            elif slot == 2:
                away_points[2].append(p['act'])
                #bisect.insort(points[2], p['act'])
            elif slot == 3:
                away_points[4].append(p['act'])
                #bisect.insort(points[4], p['act'])
            elif slot == 4:
                away_points[6].append(p['act'])
               # bisect.insort(points[6], p['act'])
            elif slot == 15:
                away_points[15].append(p['act'])
                #bisect.insort(points[15], p['act'])
            elif slot == 5:
                away_points[17].append(p['act'])
                #bisect.insort(points[17], p['act'])
    # variable to hold the running point total for a perfect lineup
    prfPoints = 0
    # variable to hold the RB, WR, and TE points that did not get into the perfect starting lineup,
    # the max point in this line up will be the starting flex player
    flex =[]
    for i, v in away_points.items():
        scores = list(sorted(v, reverse=True))
        if i == 0:
            prfPoints += scores[0]
        elif i == 2:
            prfPoints += sum(scores[0:2])
            flex.append(scores[2:])
        elif i == 4:
            prfPoints += sum(scores[0:2])
            flex.append(scores[2:])
        elif i == 6:
            prfPoints += scores[0]
            flex.append(scores[1:])
        elif i == 15:
            prfPoints += scores[0]
        elif i == 17:
            prfPoints += scores[0]
        
    #flattens the list of list created from the flex list generation. the max value is added to the perfect lineup
    prfPoints += sorted([item for sublist in flex for item in sublist], reverse=True)[0]
    #calculates the wasted points
    data[away_team][week]['wastedPts'] = prfPoints - data[away_team][week]['pf']
    # marks a perfect lineup if there are no wasted points
    if data[away_team][week]['wastedPts'] == 0:
        data[away_team][week]['prfLineup'] == 1
    #adds perfect lineup score to the team data for the week
    data[away_team][week]['prfScore'] = prfPoints

    # calculates the running totals for act and proj pf/pa
    if week == 1:
        data[away_team][week]['pfTotal'] = data[away_team][week]['pf']
        data[away_team][week]['paTotal'] = data[away_team][week]['pa']
        data[away_team][week]['ppfTotal'] = data[away_team][week]['ppf']
        data[away_team][week]['ppaTotal'] = data[away_team][week]['ppa']
    else:
        data[away_team][week]['pfTotal'] += data[away_team][week-1]['pf']
        data[away_team][week]['paTotal'] += data[away_team][week-1]['pa']
        data[away_team][week]['ppfTotal'] += data[away_team][week-1]['ppf']
        data[away_team][week]['ppaTotal'] += data[away_team][week-1]['ppa']
    # saves the ESPN power ranking
    data[away_team][week]['pwrESPN'] = league_json['teams'][away_team-1]['currentProjectedRank']   
                    
    ### calculating perfect line up scores
    # creating a dictionary for the HOME team points by position id
    home_points =  {0:[],2:[],4:[], 6:[], 15:[],17:[]}
    for pos in [0, 2, 4, 6 ,15, 17, 20]:
        for i, p in data[home_team][week]['roster'][pos].items():
            slot = p['defaultSlot']
            if slot == 1:
                home_points[0].append(p['act'])
            elif slot == 2:
                home_points[2].append(p['act'])
                #bisect.insort(points[2], p['act'])
            elif slot == 3:
                home_points[4].append(p['act'])
                #bisect.insort(points[4], p['act'])
            elif slot == 4:
                home_points[6].append(p['act'])
                # bisect.insort(points[6], p['act'])
            elif slot == 15:
                home_points[15].append(p['act'])
                #bisect.insort(points[15], p['act'])
            elif slot == 5:
                home_points[17].append(p['act'])
                #bisect.insort(points[17], p['act'])
        
    # variable to hold the running point total for a perfect lineup
    prfPoints = 0
    # variable to hold the RB, WR, and TE points that did not get into the perfect starting lineup,
    # the max point in this line up will be the starting flex player
    flex =[]
    for i, v in home_points.items():
        scores = list(sorted(v, reverse=True))
        if i == 0:
            prfPoints += scores[0]
        elif i == 2:
            prfPoints += sum(scores[0:2])
            flex.append(scores[2:])
        elif i == 4:
            prfPoints += sum(scores[0:2])
            flex.append(scores[2:])
        elif i == 6:
            prfPoints += scores[0]
            flex.append(scores[1:])
        elif i == 15:
            prfPoints += scores[0]
        elif i == 17:
            prfPoints += scores[0]
                
    #flattens the list of list created from the flex list generation. the max value is added to the perfect lineup
    prfPoints += sorted([item for sublist in flex for item in sublist], reverse=True)[0]
    #calculates the wasted points
    data[home_team][week]['wastedPts'] = prfPoints - data[home_team][week]['pf']
    # marks a perfect lineup if there are no wasted points
    if data[home_team][week]['wastedPts'] == 0:
        data[home_team][week]['prfLineup'] == 1
    #adds perfect lineup score to the team data for the week
    data[home_team][week]['prfScore'] = prfPoints

    # calculates the running totals for act and proj pf/pa
    if week == 1:
        data[home_team][week]['pfTotal'] = data[home_team][week]['pf']
        data[home_team][week]['paTotal'] = data[home_team][week]['pa']
        data[home_team][week]['ppfTotal'] = data[home_team][week]['ppf']
        data[home_team][week]['ppaTotal'] = data[home_team][week]['ppa']
    else:
        data[home_team][week]['pfTotal'] += data[home_team][week-1]['pf']
        data[home_team][week]['paTotal'] += data[home_team][week-1]['pa']
        data[home_team][week]['ppfTotal'] += data[home_team][week-1]['ppf']
        data[home_team][week]['ppaTotal'] += data[home_team][week-1]['ppa']
    # saves the ESPN power ranking
    data[home_team][week]['pwrESPN'] = league_json['teams'][home_team-1]['currentProjectedRank']
    
week_scores = sorted(week_scores)
for team in data:
    data[team][week]['scoreRnk'] = week_scores.index(data[team][week]["pf"]) + 1
    data[team][week]['oppScoreRnk'] = week_scores.index(data[data[team][week]['oppId']][week]["pf"]) + 1


In [ ]:
data[1]

In [121]:
# team game info and players on a team roster
filepath_boxscore = "..\\data\\Season\\2023week1_boxscore.txt"
# This has team data
filepath_league = "..\\data\\Season\\2023week1_league.txt"
# players is all nfl players infor for the week
#filepath_player = "..\\data\\Season\\2021week2_player.txt"

In [95]:
# open text, convert to json
with open(filepath_boxscore) as json_file:
    js = json.load(json_file)

# save json file     
#with open('data.json', 'w', encoding='utf-8') as f:
#    json.dump(js, f, ensure_ascii=False, indent=4)

In [130]:
headers = {
  'X-Fantasy-Filter': '{"players": {"limit": 5000, "sortPercOwned":{"sortAsc":false,"sortPriority":1}}}'
}
cookies = {'SWID':'{53904938-E572-4FD1-9F30-10A8A39DA775}', 
'espn_s2':'AEBYlwRe6QId5pjWbC23XMeT1GfxZRz4Im3RWWAidsQjflJzttPKwIigRhW1mR6yHfVCfbXSonbsX%2B15VCIGu6mTFw6FajidlwzLIUk3Np9%2Bl7I2VNiaxPh%2F%2FUg0jHs6uoko6VTPs2Y1GlgMMOJ8IRVhtJjW4RIyrbyO%2Ftrvf3HWmVvhMZGdG3EKO9h4UYbcUaVIwZxUBbmrKKmFOZoHaMVpgXGGAJVkhtgxA2RVgW%2FqOkQhoPopTDvRuMTJuTEiOghEDxRREO9ztYr5S%2BhmByiQ'
}
url = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/{}/segments/0/leagues/245118?view=mDraftDetail&view=mLiveScoring&view=mMatchupScore&view=mPendingTransactions&view=mPositionalRatings&view=mSettings&view=mTeam&view=modular&view=mNav&view=mMatchupScore"
response = requests.request("GET", url.format(season), headers=headers, cookies=cookies)
league_json = json.loads(response.content)

In [136]:
# load boxscore from week 1 to finish the data
with open('..\\data\\Season\\{}week{}_boxscore.txt'.format(season, week)) as json_file:
    boxscore_json = json.load(json_file)

with open('..\\data\\Season\\{}week{}_league.txt'.format(season, week)) as json_file:
    league_json = json.load(json_file)

In [138]:
league_json['teams'][0]

{'abbrev': 'jb',
 'currentProjectedRank': 9,
 'divisionId': 0,
 'draftDayProjectedRank': 3,
 'draftStrategy': {'futureKeeperPlayerIds': [], 'keeperPlayerIds': [4362628]},
 'id': 1,
 'isActive': False,
 'location': 'Ayahuantawin',
 'logo': 'https://images.halloween.com/products/56553/2-1-137546/spyro-the-dragon-adult-jumpsuit-costume-alt2.jpg',
 'logoType': 'CUSTOM_VALID',
 'name': "Chase'n the dragon",
 'nickname': 'Tea',
 'owners': ['{53904938-E572-4FD1-9F30-10A8A39DA775}'],
 'playoffSeed': 12,
 'points': 0.0,
 'pointsAdjusted': 0.0,
 'pointsDelta': 0.0,
 'primaryOwner': '{53904938-E572-4FD1-9F30-10A8A39DA775}',
 'rankCalculatedFinal': 0,
 'rankFinal': 0,
 'record': {'away': {'gamesBack': 0.0,
   'losses': 0,
   'percentage': 0.0,
   'pointsAgainst': 0.0,
   'pointsFor': 0.0,
   'streakLength': 0,
   'streakType': 'NONE',
   'ties': 0,
   'wins': 0},
  'division': {'gamesBack': 0.0,
   'losses': 0,
   'percentage': 0.0,
   'pointsAgainst': 0.0,
   'pointsFor': 0.0,
   'streakLength': 